# INTRODUCCION A MONGODB

MongoDB es una base de datos basada en documentos con un esquema de datos dinámico. La notación de objetos de JavaScript (JSON) que admite es una opción natural para trabajar con objetos en lenguajes de programación modernos como JavaScript, Python y otros. Esto proporciona una alternativa a los sistemas de administración de bases de datos relacionales (RDBMS) más tradicionales, como SQL. MongoDB es un ejemplo de una base de datos NoSQL. Estas bases de datos suelen utilizar colecciones de documentos en lugar de las tablas utilizadas en RDBMS. Estas bases de datos admiten esquemas de bases de datos dinámicas, lo que las hace sensibles a los cambios en la estructura de los datos.

Este breve manual proporciona algunos ejemplos del uso de MongoDB con Python utilizando la biblioteca pymongo. Esto concluye presentando mongoengine para trabajar con bases de datos Mongo en proyectos de software modernos y cómo convertirlos en objetos de datos para su posterior análisis.

### ESTRUCTURA

La estructura de una base de datos de documentos difiere de la de una base de datos relacional que almacena datos en filas (registros) y columnas (campos).

![RDBMS](https://miro.medium.com/max/720/1*krhmL3000bPZeWYDZLjoOQ.png) 

En esta estructura cada columna solo debe contener datos del mismo tipo. Por ejemplo, solo esperaríamos ver datos de artículos en stock en la columna de artículos en stock. Cualquier dato nuevo requeriría crear una nueva columna o una nueva tabla y luego definir una relación entre las tablas utilizando un identificador único (una clave principal) denominado clave externa en las tablas posteriores. Cambiar la estructura de estos datos, especialmente cuando ya contienen datos, puede ser más complejo y puede requerir el uso de herramientas de migración.

En contraste con esto, MongoDB almacena datos como colecciones de documentos utilizando pares clave/valor:

![RDBMS](https://miro.medium.com/max/602/1*LZ78OSSpNLpXsMmxkQV3Hw.png) 

A diferencia de las bases de datos relacionales en las que tendríamos que crear una nueva columna en una tabla para almacenar información, los datos se pueden incrustar. Esto significa que solo necesitamos almacenar lo que es relevante en lugar de crear redundancia.

In [45]:
import pymongo
from urllib.parse import quote_plus
import datetime
import pprint as pp
import pandas as pd

In [33]:
user = "schedulin"
passw = "MySQLIsFun"
host = "mongodb-schedulin.hlbmw0g.mongodb.net"
database = "schedulin"

### CONEXIONES

In [4]:
def connect():
    client = pymongo.MongoClient(
        "mongodb+srv://{0}:{1}@{2}/?retryWrites=true&w=majority" \
            .format(user, passw, host))
    print("Connected to the MongoDB database!\nParameters: {0}" \
        .format(client))
    db = client[database]
    return db

In [5]:
db = connect()

Connected to the MongoDB database!
Parameters: MongoClient(host=['ac-gshrp3d-shard-00-01.hlbmw0g.mongodb.net:27017', 'ac-gshrp3d-shard-00-00.hlbmw0g.mongodb.net:27017', 'ac-gshrp3d-shard-00-02.hlbmw0g.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-id34b1-shard-0', tls=True)


#### COLECCIONES

Podemos agregar una colección a la base de datos. Cada base de datos puede contener varias colecciones.



In [6]:
users = db.users
reservations = db.reservations

#### DOCUMENTOS

Luego podemos agregar algunos datos (un documento) a la colección. Digamos que queremos almacenar algunos detalles básicos sobre un usuario. Esto podría incluir su nombre, edad, .... También podemos almacenar su presión edad, que generalmente se muestra con números. En MongoDB, los campos (elementos de datos) se encapsulan entre llaves ({}) utilizando la notación de objetos de JavaScript. Cada campo consta de un par clave/valor. El nombre del campo (clave) está entre comillas seguido de dos puntos y luego el valor relacionado. Los valores textuales (datos de texto) también se encapsulan entre comillas, los números (datos numéricos) no. Los valores también pueden contener otros objetos y matrices. Las matrices pueden almacenar listas de datos y otros pares de valores clave y se indican con corchetes ([]). Aquí podemos almacenar las claves y valores para el bloque, portal, piso y puerta junto con los valores de los datos.

Hay que tener en cuenta que MongoDB agrega automáticamente un ObjectId para identificar de forma única cada documento. Esta es una cadena hexadecimal de 12 bytes que consta de una marca de tiempo, un valor generado aleatoriamente y un contador incremental. Estas identificaciones se muestran cuando se envían datos. También puede anular esto proporcionando sus propios valores para el campo "_id" si es necesario.


In [31]:
user = {
    "name": "Mike Platinas",
    "email": "mike.platinas@mail.com",
    "status": "ACTIVE",
    "interests": [
        "SWIMMING", "CYCLING", 
    ],
    "creation_date": datetime.datetime.utcnow(),
    "location": {
        "block": "B",
        "gate": "C",
        "floor": 2,
        "door": "C"}}
user_id = users.insert_one(user).inserted_id
print(user_id)

6366b29cc5b6129baf54cfef


In [32]:
user = {
    "name": "Jonh Doe",
    "email": "john.doe@mail.com",
    "status": "ACTIVE",
    "interests": [
        "PADELING", "CYCLING", 
    ],
    "creation_date": datetime.datetime.utcnow(),
    "location": {
        "block": "B",
        "gate": "A",
        "floor": 7,
        "door": "A"}}
user_id = users.insert_one(user).inserted_id
print(user_id)

6366b2b2c5b6129baf54cff0


In [33]:
user = {
    "name": "Jake Sparrow",
    "email": "jake.sparrow@mail.com",
    "status": "INACTIVE",
    "creation_date": datetime.datetime.utcnow(),
    "location": {
        "block": "A",
        "gate": "C",
        "floor": 1,
        "door": "A"}}
user_id = users.insert_one(user).inserted_id
print(user_id)

6366b2c9c5b6129baf54cff1


In [21]:
type(user_id)

bson.objectid.ObjectId

In [22]:
str_user_id = str(user_id)

In [23]:
pp.pprint(users.find_one({"_id": str_user_id}))

None


#### FIND ONE DOCUMENT

Ver los datos de esta manera hace que sea bastante difícil de leer, especialmente si tiene muchos campos y documentos para generar. Afortunadamente, Python tiene una bonita biblioteca de impresión para tal propósito. Si modificamos el código para importar la biblioteca y usamos la función de pprint.

In [24]:
from bson.objectid import ObjectId

pp.pprint(users.find_one({"_id": ObjectId(str_user_id)}))

{'_id': ObjectId('63650237ec395898e8884e11'),
 'creation_date': datetime.datetime(2022, 11, 4, 12, 14, 47, 638000),
 'email': 'mike.platinas@mail.com',
 'name': 'Mike Platinas',
 'status': 'ACTIVE'}


In [18]:
pp.pprint(users.find_one({"_id": user_id}))

{'_id': ObjectId('63650237ec395898e8884e11'),
 'creation_date': datetime.datetime(2022, 11, 4, 12, 14, 47, 638000),
 'email': 'mike.platinas@mail.com',
 'name': 'Mike Platinas',
 'status': 'ACTIVE'}


#### SHOW COLLECTION LIST

In [25]:
print(db.list_collection_names())

['users']


#### CREATE MULTIPLE DOCUMENTS

In [7]:
reservation_lst = [
    {"email": "mike.platinas@mail.com",
    "resource": "PADEL",
    "timetables": {
        "start_time": "11:00", 
        "end_time": "12:30"},
    "price": 3,
    "date": datetime.datetime(year = 2022, month = 10, day = 31),
    "creation_date": datetime.datetime.utcnow()},
    {"email": "mike.platinas@mail.com",
    "resource": "PADEL",
    "timetables": {
        "start_time": "12:30", 
        "end_time": "14:00"},
    "price": 5,
    "date": datetime.datetime(year = 2022, month = 10, day = 31),
    "creation_date": datetime.datetime.utcnow()},
    {"email": "john.doe@mail.com",
    "resource": "PISCINA",
    "timetables": {
        "start_time": "16:00"},
    "date": datetime.datetime(year = 2022, month = 11, day = 1),
    "creation_date": datetime.datetime.utcnow()}]
result = reservations.insert_many(reservation_lst)
for result in result.inserted_ids:
    print(result)

6366b871a72f5f099a4f6765
6366b871a72f5f099a4f6766
6366b871a72f5f099a4f6767


In [6]:
print(db.list_collection_names())

['users', 'reservations']


#### COUNT DOCUMENTS IN A COLLECTION

In [16]:
reservations.count_documents({})

4

In [17]:
reservations.count_documents({"resource": "PADEL"})

2

In [10]:
reservation = {
    "email": "mike.platinas@mail.com",
    "resource": "PISCINA",
    "timetables": {
        "start_time": "16:00"},
    "price": 1,
    "date": datetime.datetime(year = 2022, month = 10, day = 31),
    "creation_date": datetime.datetime.utcnow()}  
reservation_id = reservations.insert_one(reservation).inserted_id
print(reservation_id)

6366b97aa72f5f099a4f6769


In [38]:
reservations.count_documents({})

3

#### QUERIES

Hay varios métodos para consultar datos. Todos los métodos usan la función find(). Se puede proporcionar una consulta seguida del campo o campos que desea devolver en el formulario del tipo: **colección.find({ < consulta > }, { < campos > })**.

![OPERATORS](https://miro.medium.com/max/640/1*S4uhMlJ173rK_qOoN25j8Q.png)  
![LOGICALS](https://miro.medium.com/max/640/1*uR9igCWQyngNY0cNnLpMJg.png)

In [39]:
for reservation in reservations.find():
    pp.pprint(reservation)

{'_id': ObjectId('6365240b049404598640efb1'),
 'creation_date': datetime.datetime(2022, 11, 4, 14, 39, 7, 389000),
 'date': datetime.datetime(2022, 10, 31, 0, 0),
 'email': 'mike.platinas@mail.com',
 'resource': 'PADEL',
 'timetables': {'end_time': '12:30', 'start_time': '11:00'}}
{'_id': ObjectId('6365240b049404598640efb2'),
 'creation_date': datetime.datetime(2022, 11, 4, 14, 39, 7, 389000),
 'date': datetime.datetime(2022, 10, 31, 0, 0),
 'email': 'mike.platinas@mail.com',
 'resource': 'PADEL',
 'timetables': {'end_time': '14:00', 'start_time': '12:30'}}
{'_id': ObjectId('6365240b049404598640efb3'),
 'creation_date': datetime.datetime(2022, 11, 4, 14, 39, 7, 389000),
 'date': datetime.datetime(2022, 11, 1, 0, 0),
 'email': 'mike.platinas@mail.com',
 'resource': 'PISCINA',
 'timetables': {'start_time': '16:00'}}


In [21]:
for reservation in reservations.find().sort("timetables.end_time", -1).limit(2):
    pp.pprint(reservation)

{'_id': ObjectId('6366b871a72f5f099a4f6766'),
 'creation_date': datetime.datetime(2022, 11, 5, 19, 24, 33, 878000),
 'date': datetime.datetime(2022, 10, 31, 0, 0),
 'email': 'mike.platinas@mail.com',
 'price': 5,
 'resource': 'PADEL',
 'timetables': {'end_time': '14:00', 'start_time': '12:30'}}
{'_id': ObjectId('6366b871a72f5f099a4f6765'),
 'creation_date': datetime.datetime(2022, 11, 5, 19, 24, 33, 878000),
 'date': datetime.datetime(2022, 10, 31, 0, 0),
 'email': 'mike.platinas@mail.com',
 'price': 3,
 'resource': 'PADEL',
 'timetables': {'end_time': '12:30', 'start_time': '11:00'}}


In [11]:
for reservation in reservations.find({"resource": "PISCINA"}):
    pp.pprint(reservation)

{'_id': ObjectId('63651a71049404598640efb0'),
 'creation_date': datetime.datetime(2022, 11, 4, 13, 58, 9, 5000),
 'date': datetime.datetime(2022, 11, 1, 0, 0),
 'email': 'mike.platinas@mail.com',
 'resource': 'PISCINA',
 'timetables': ['16:00', '22:00']}


In [14]:
d = datetime.datetime(2022, 11, 1)
for reservation in reservations.find({"date": {"$lt": d}}).sort("author"):
    pp.pprint(reservation)

{'_id': ObjectId('63651a71049404598640efae'),
 'creation_date': datetime.datetime(2022, 11, 4, 13, 58, 9, 5000),
 'date': datetime.datetime(2022, 10, 31, 0, 0),
 'email': 'mike.platinas@mail.com',
 'resource': 'PADEL',
 'timetables': ['11:00', '12:30']}
{'_id': ObjectId('63651a71049404598640efaf'),
 'creation_date': datetime.datetime(2022, 11, 4, 13, 58, 9, 5000),
 'date': datetime.datetime(2022, 10, 31, 0, 0),
 'email': 'mike.platinas@mail.com',
 'resource': 'PADEL',
 'timetables': ['12:30', '14:00']}


In [57]:
for user in users.find():
    pp.pprint(user)

{'_id': ObjectId('636524f3049404598640efb4'),
 'creation_date': datetime.datetime(2022, 11, 4, 14, 42, 59, 354000),
 'email': 'mike.platinas@mail.com',
 'interests': ['SWIMMING', 'CYCLING'],
 'name': 'Mike Platinas',
 'status': 'ACTIVE'}
{'_id': ObjectId('63652538049404598640efb5'),
 'creation_date': datetime.datetime(2022, 11, 4, 14, 44, 8, 663000),
 'email': 'john.doe@mail.com',
 'interests': ['PADELING', 'CYCLING'],
 'name': 'Jonh Doe',
 'status': 'ACTIVE'}
{'_id': ObjectId('6365271b049404598640efb6'),
 'creation_date': datetime.datetime(2022, 11, 4, 14, 52, 11, 343000),
 'email': 'jake.sparrow@mail.com',
 'name': 'Jake Sparrow',
 'status': 'INACTIVE'}


In [17]:
d = datetime.datetime(2022, 10, 31)
for reservation in reservations.find({"date": {"$gt": d}}):
    pp.pprint(reservation)

{'_id': ObjectId('6365240b049404598640efb3'),
 'creation_date': datetime.datetime(2022, 11, 4, 14, 39, 7, 389000),
 'date': datetime.datetime(2022, 11, 1, 0, 0),
 'email': 'john.doe@mail.com',
 'resource': 'SWIMMING POOL',
 'timetables': {'start_time': '16:00'}}


In [18]:
d = datetime.datetime(2022, 10, 31)
for reservation in reservations.find(
    {"date": {"$gt": d}}, 
    {"email"}):
    pp.pprint(reservation)

{'_id': ObjectId('6365240b049404598640efb3'), 'email': 'john.doe@mail.com'}


In [19]:
d = datetime.datetime(2022, 10, 31)
for reservation in reservations.find(
    {"date": {"$gt": d}}, 
    {"timetables.start_time"}):
    pp.pprint(reservation)

{'_id': ObjectId('6365240b049404598640efb3'),
 'timetables': {'start_time': '16:00'}}


In [27]:
d = datetime.datetime(2022, 10, 30)
for reservation in reservations.find({
     "$and" : [
        {"date": {"$gt": d}},
        {"timetables.end_time": {"$lt": '14:30'}}]},
    {"timetables.start_time", "email", "date"}
        
        ):
    pp.pprint(reservation)

{'_id': ObjectId('6365240b049404598640efb1'),
 'date': datetime.datetime(2022, 10, 31, 0, 0),
 'email': 'mike.platinas@mail.com',
 'timetables': {'start_time': '11:00'}}
{'_id': ObjectId('6365240b049404598640efb2'),
 'date': datetime.datetime(2022, 10, 31, 0, 0),
 'email': 'mike.platinas@mail.com',
 'timetables': {'start_time': '12:30'}}


#### Agregaciones

A menudo, cuando trabajamos con datos, no solo queremos extraer subconjuntos de datos mediante consultas, sino que queremos producir nueva información a partir de los datos existentes. Esto a menudo implica realizar varios cálculos, como encontrar el promedio o la suma de algún valor. Por ejemplo, el salario medio de los empleados.  

Pasamos una matriz a la función agregada. El parámetro **$unwind** se usa para deconstruir la matriz de calificaciones para generar un documento para cada elemento. A continuación, usamos el parámetro **$match** que incluye todo (usando llaves de apertura y cierre). Podríamos filtrar más aquí proporcionando criterios adicionales. A continuación, usamos el parámetro **$group** para agrupar los datos a los que queremos aplicar el cálculo. Finalmente, creamos una nueva clave a la que aplicamos un parámetro como puede ser **$avg** entre otros.

In [16]:
pipeline = [
    {"$unwind": "$price"},
    {"$group": {"_id": {"resource": "$resource"}, "Avg": {"$avg": "$price"}}},
    {"$sort": {"Avg": 1}}
]
for r in list(db.reservations.aggregate(pipeline)):
    pp.pprint(r)

{'Avg': 1.0, '_id': {'resource': 'PISCINA'}}
{'Avg': 4.0, '_id': {'resource': 'PADEL'}}


In [14]:
pipeline = [
    {"$match" : { "resource": "PADEL"}},
    {"$unwind": "$price"},
    {"$group": {"_id": {"resource": "$resource"}, "Avg": {"$avg": "$price"}}},
]
for r in list(db.reservations.aggregate(pipeline)):
    pp.pprint(r)

{'Avg': 4.0, '_id': {'resource': 'PADEL'}}


In [73]:
pipeline = [
    {"$match" : { "status": "ACTIVE"}},
    {"$unwind": "$interests"},
    {"$group": {"_id": {"name": "$name"}, "Main interest": {"$first": "$interests"}}},
    {"$project": {"_id": 1, "Main interest": 1}}
]
for r in list(db.users.aggregate(pipeline)):
    pp.pprint(r)

{'Main interest': 'SWIMMING', '_id': {'name': 'Mike Platinas'}}
{'Main interest': 'PADELING', '_id': {'name': 'Jonh Doe'}}


In [14]:
pipeline = [
    {"$match" : {"$and" : [{"status": "ACTIVE"},{"status": "INACTIVE"}]}},
    {"$unwind": "$interests"},
    {"$group": {"_id": {"name": "$name"}, "Main interest": {"$first": "$interests"}}},
    {"$project": {"_id": 1, "Main interest": 1}}
]
for r in list(db.users.aggregate(pipeline)):
    pp.pprint(r)

In [76]:
from bson.son import SON

pipeline = [
    {"$unwind": '$timetables' },
    {"$group": {"_id": '$timetables.start_time', "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1), ("_id", -1)])},
]
pp.pprint(list(db.reservations.aggregate(pipeline)))

[{'_id': '16:00', 'count': 2},
 {'_id': '12:30', 'count': 1},
 {'_id': '11:00', 'count': 1}]


#### ACTUALIZACIONES DE DATOS

También es posible que deseemos actualizar los datos que hemos agregado previamente a una colección. De nuevo, podemos actualizar uno o varios registros. Digamos que agregamos accidentalmente un email erroneo a un usuario. Aquí podemos usar la función update_one pasando el campo que queremos actualizar buscando el par clave/valor y luego usamos la opción $set (precedida por un signo de dólar) especificando la clave y el nuevo valor . Esto sobrescribirá el valor inicial con el nuevo.

In [83]:
users = db.users
for user in users.find({"email": "jake.sparrow@mail.com"}):
    pp.pprint(user)

{'_id': ObjectId('6365271b049404598640efb6'),
 'creation_date': datetime.datetime(2022, 11, 4, 14, 52, 11, 343000),
 'email': 'jake.sparrow@mail.com',
 'name': 'Jake Sparrow',
 'status': 'INACTIVE'}


In [84]:
to_search = {"email": "jake.sparrow@mail.com"}
to_change = {"$set": {
    "name": "Jack Horton",
    "email": "jack.horton@mail.com"}}

users.update_one(to_search, to_change)

In [85]:
users = db.users
for user in users.find({"email": "jake.sparrow@mail.com"}):
    pp.pprint(user)

In [95]:
reservations = db.reservations
for reservation in reservations.find({"resource": {"$regex": "^PISCINA"}}):
    pp.pprint(reservation)

{'_id': ObjectId('6365240b049404598640efb3'),
 'creation_date': datetime.datetime(2022, 11, 4, 14, 39, 7, 389000),
 'date': datetime.datetime(2022, 11, 1, 0, 0),
 'email': 'john.doe@mail.com',
 'resource': 'PISCINA',
 'timetables': {'start_time': '16:00'}}
{'_id': ObjectId('63652936049404598640efb7'),
 'creation_date': datetime.datetime(2022, 11, 4, 15, 1, 10, 167000),
 'date': datetime.datetime(2022, 10, 31, 0, 0),
 'email': 'mike.platinas@mail.com',
 'resource': 'PISCINA',
 'timetables': {'start_time': '16:00'}}


In [96]:
to_search = {"resource": {"$regex": "^PISCINA"}}
to_change = {"$set": {"resource": "SWIMMING POOL"}}

num = reservations.update_many(to_search, to_change)
print(num.modified_count, " documents updated.")

2  documents updated.


In [97]:
reservations = db.reservations
for reservation in reservations.find({"resource": {"$regex": "^SWIM"}}):
    pp.pprint(reservation)

{'_id': ObjectId('6365240b049404598640efb3'),
 'creation_date': datetime.datetime(2022, 11, 4, 14, 39, 7, 389000),
 'date': datetime.datetime(2022, 11, 1, 0, 0),
 'email': 'john.doe@mail.com',
 'resource': 'SWIMMING POOL',
 'timetables': {'start_time': '16:00'}}
{'_id': ObjectId('63652936049404598640efb7'),
 'creation_date': datetime.datetime(2022, 11, 4, 15, 1, 10, 167000),
 'date': datetime.datetime(2022, 10, 31, 0, 0),
 'email': 'mike.platinas@mail.com',
 'resource': 'SWIMMING POOL',
 'timetables': {'start_time': '16:00'}}


#### JOINS o LOOKUPS

In [16]:
reservations = db.reservations

pipeline = [
    {
        '$lookup': {
            'from': 'users', 
            'localField': 'email', 
            'foreignField': 'email', 
            'as': 'user'}},
    {
        '$match':
                 {'resource' : 'PADEL',}}
]

for r in reservations.aggregate(pipeline):
    pp.pprint(r)

{'_id': ObjectId('6365240b049404598640efb1'),
 'creation_date': datetime.datetime(2022, 11, 4, 14, 39, 7, 389000),
 'date': datetime.datetime(2022, 10, 31, 0, 0),
 'email': 'mike.platinas@mail.com',
 'resource': 'PADEL',
 'timetables': {'end_time': '12:30', 'start_time': '11:00'},
 'user': [{'_id': ObjectId('636524f3049404598640efb4'),
           'creation_date': datetime.datetime(2022, 11, 4, 14, 42, 59, 354000),
           'email': 'mike.platinas@mail.com',
           'interests': ['SWIMMING', 'CYCLING'],
           'name': 'Mike Platinas',
           'status': 'ACTIVE'}]}
{'_id': ObjectId('6365240b049404598640efb2'),
 'creation_date': datetime.datetime(2022, 11, 4, 14, 39, 7, 389000),
 'date': datetime.datetime(2022, 10, 31, 0, 0),
 'email': 'mike.platinas@mail.com',
 'resource': 'PADEL',
 'timetables': {'end_time': '14:00', 'start_time': '12:30'},
 'user': [{'_id': ObjectId('636524f3049404598640efb4'),
           'creation_date': datetime.datetime(2022, 11, 4, 14, 42, 59, 354000),


#### INDEXES

In [25]:
reservations.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]}}

In [26]:
pp.pprint(reservations.find({'resource':'PADEL'}).explain()['executionStats'])

{'allPlansExecution': [],
 'executionStages': {'advanced': 2,
                     'direction': 'forward',
                     'docsExamined': 4,
                     'executionTimeMillisEstimate': 0,
                     'filter': {'resource': {'$eq': 'PADEL'}},
                     'isEOF': 1,
                     'nReturned': 2,
                     'needTime': 3,
                     'needYield': 0,
                     'restoreState': 0,
                     'saveState': 0,
                     'stage': 'COLLSCAN',
                     'works': 6},
 'executionSuccess': True,
 'executionTimeMillis': 0,
 'nReturned': 2,
 'totalDocsExamined': 4,
 'totalKeysExamined': 0}


In [27]:
reservations.create_index('resource')

'resource_1'

In [28]:
reservations.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'resource_1': {'v': 2, 'key': [('resource', 1)]}}

In [29]:
pp.pprint(reservations.find({'resource':'PADEL'}).explain()['executionStats'])

{'allPlansExecution': [],
 'executionStages': {'advanced': 2,
                     'alreadyHasObj': 0,
                     'docsExamined': 2,
                     'executionTimeMillisEstimate': 0,
                     'inputStage': {'advanced': 2,
                                    'direction': 'forward',
                                    'dupsDropped': 0,
                                    'dupsTested': 0,
                                    'executionTimeMillisEstimate': 0,
                                    'indexBounds': {'resource': ['["PADEL", '
                                                                 '"PADEL"]']},
                                    'indexName': 'resource_1',
                                    'indexVersion': 2,
                                    'isEOF': 1,
                                    'isMultiKey': False,
                                    'isPartial': False,
                                    'isSparse': False,
                       

URL de consulta: https://towardsdatascience.com/using-mongo-databases-in-python-e93bc3b6ff5f

In [30]:
!pip3 install mongoengine

     |████████████████████████████████| 108 kB 10.6 MB/s 


In [38]:
import mongoengine

mongoengine.disconnect()
mongoengine.connect(host = "mongodb://{0}:{1}@{2}:27017/{3}" \
    .format(user, passw, host, database))

MongoClient(host=['mongodb-schedulin.hlbmw0g.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

In [49]:
class User(mongoengine.Document):
    user_id = StringField(required = True)
    name = StringField()
    status = StringField()
    interests = ListField()
    creation_date = DateTimeField()
    location = DictField()

In [41]:
users_data = users.find()
users_lst = list(users_data)
users_lst

[{'_id': ObjectId('6366b29cc5b6129baf54cfef'),
  'name': 'Mike Platinas',
  'email': 'mike.platinas@mail.com',
  'status': 'ACTIVE',
  'interests': ['SWIMMING', 'CYCLING'],
  'creation_date': datetime.datetime(2022, 11, 5, 18, 59, 40, 791000),
  'location': {'block': 'B', 'gate': 'C', 'floor': 2, 'door': 'C'}},
 {'_id': ObjectId('6366b2b2c5b6129baf54cff0'),
  'name': 'Jonh Doe',
  'email': 'john.doe@mail.com',
  'status': 'ACTIVE',
  'interests': ['PADELING', 'CYCLING'],
  'creation_date': datetime.datetime(2022, 11, 5, 19, 0, 2, 944000),
  'location': {'block': 'B', 'gate': 'A', 'floor': 7, 'door': 'A'}},
 {'_id': ObjectId('6366b2c9c5b6129baf54cff1'),
  'name': 'Jake Sparrow',
  'email': 'jake.sparrow@mail.com',
  'status': 'INACTIVE',
  'creation_date': datetime.datetime(2022, 11, 5, 19, 0, 25, 294000),
  'location': {'block': 'A', 'gate': 'C', 'floor': 1, 'door': 'A'}}]

In [50]:
users_df = pd.DataFrame.from_dict(users_lst)

In [51]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   _id            3 non-null      object        
 1   name           3 non-null      object        
 2   email          3 non-null      object        
 3   status         3 non-null      object        
 4   interests      2 non-null      object        
 5   creation_date  3 non-null      datetime64[ns]
 6   location       3 non-null      object        
dtypes: datetime64[ns](1), object(6)
memory usage: 296.0+ bytes


In [52]:
users_df.head()

,_id,name,email,status,interests,creation_date,location
0,6366b29cc5b6129baf54cfef,Mike Platinas,mike.platinas@mail.com,ACTIVE,"[SWIMMING, CYCLING]",2022-11-05 18:59:40.791,"{'block': 'B', 'gate': 'C', 'floor': 2, 'door'..."
1,6366b2b2c5b6129baf54cff0,Jonh Doe,john.doe@mail.com,ACTIVE,"[PADELING, CYCLING]",2022-11-05 19:00:02.944,"{'block': 'B', 'gate': 'A', 'floor': 7, 'door'..."
2,6366b2c9c5b6129baf54cff1,Jake Sparrow,jake.sparrow@mail.com,INACTIVE,NaN,2022-11-05 19:00:25.294,"{'block': 'A', 'gate': 'C', 'floor': 1, 'door'..."


In [55]:
users_df[users_df["status"] == 'ACTIVE']

,_id,name,email,status,interests,creation_date,location
0,6366b29cc5b6129baf54cfef,Mike Platinas,mike.platinas@mail.com,ACTIVE,"[SWIMMING, CYCLING]",2022-11-05 18:59:40.791,"{'block': 'B', 'gate': 'C', 'floor': 2, 'door'..."
1,6366b2b2c5b6129baf54cff0,Jonh Doe,john.doe@mail.com,ACTIVE,"[PADELING, CYCLING]",2022-11-05 19:00:02.944,"{'block': 'B', 'gate': 'A', 'floor': 7, 'door'..."
